In [ ]:
#SMART AGENT RECRUITMENT

#Sami Akhtar

#ANALYSIS AND DESCRIPTION

#This is a classification machine learning problem to identify the best agents / applicants, for a Financial Distribution company, who will be able to source business for the company within 3 months post their 7 day corporate training.


#I have made the predictions using LightGBM. Other models like XGBoost and AdaBoost was also used for experimentation.

!pip install lightgbm
!pip install xgboost

#Importing Libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set(style="darkgrid")

from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import roc_curve

from lightgbm import LGBMClassifier
from sklearn.ensemble import AdaBoostClassifier
from xgboost import XGBClassifier
from xgboost import plot_importance

from scipy.stats import randint as sp_randint

import warnings 
warnings.filterwarnings("ignore")

__Loading Datasets__

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

train.head()

test.head()

print("Train Dataset has {} records and {} features".format(train.shape[0], train.shape[1]))
print("Test Dataset has {} records and {} features".format(test.shape[0], test.shape[1]))

train.info()

test.info()

#1. EXPLORATORY DATA ANALYSIS (EDA) 

# 1.1. Missing Values

train.isnull().mean()

test.isnull().mean()

#There are a lot of missing values in both in **`train`** and **`test`** set. The features having missing values are :
* Applicant_City_PIN
* Applicant_Gender
* Applicant_BirthDate
* Applicant_Marital_Status
* Applicant_Occupation
* Applicant_Qualification
* Manager_DOJ
* Manager_Joining_Designation
* Manager_Current_Designation
* Manager_Grade
* Manager_Status                 
* Manager_Gender                 
* Manager_DoB                    
* Manager_Num_Application        
* Manager_Num_Coded              
* Manager_Business               
* Manager_Num_Products           
* Manager_Business2              
* Manager_Num_Products2 #

#1.2. Target Variable Distribution

#The dataset is almost balanced with the classes, 0 having **65.7 %** records and 1 having **34.3%** records.

train['Business_Sourced'].value_counts(normalize=True)

sns.countplot(x='Business_Sourced', data=train)

#1.3. Numerical Feature Distribution

#The Numerical Features in the **`train`** are :
* Manager_Num_Application
* Manager_Num_Coded
* Manager_Business
* Manager_Num_Products
* Manager_Business2
* Manager_Num_Products2

#1.3.1. Univariate Analysis

def UNI_numerical(df, num_feat):
    
    cols = len(num_feat)
    plt.figure(figsize = (7*cols,3), dpi = 100)
    
    for idx , feat in enumerate(num_feat) :
        
        # calculating descriptive statistics values of a feature
        min_val = df[feat].min()
        max_val = df[feat].max()
        ran = max_val - min_val
        
        mean_val = df[feat].mean()
        median_val = df[feat].median()
        std_dev = df[feat].std()
        skew = df[feat].skew()
        kurt = df[feat].kurtosis()
        
        # Calculating points of standard deviation
        std_points = mean_val+std_dev , mean_val-std_dev
        
        # PLotting the feature with the information
        plt.subplot(1, cols+1, idx+1)
        sns.kdeplot(x=feat, data=train, shade=True)
        sns.lineplot(x=std_points, y=[0,0], color = 'black', label = "std_dev")
        sns.scatterplot(x=[min_val,max_val], y=[0,0], color = 'orange', label = "min/max")
        sns.scatterplot(x=[mean_val], y=[0], color = 'red', label = "mean")
        sns.scatterplot(x=[median_val], y=[0], color = 'blue', label = "median")
        plt.xlabel('{}'.format(feat), fontsize = 20)
        plt.ylabel('density')
        plt.title('std_dev = {}; kurtosis = {};\nskew = {}; range = {}\nmean = {}; median = {}'.format((round(std_points[0],2),round(std_points[1],2)),
                                                                                                   round(kurt,2),
                                                                                                   round(skew,2),
                                                                                                   (round(min_val,2),round(max_val,2),round(ran,2)),
                                                                                                   round(mean_val,2),
                                                                                                   round(median_val,2)))

#__Train Dataset__

UNI_numerical(train, ['Manager_Num_Application','Manager_Num_Coded', 'Manager_Business'])

UNI_numerical(train, ['Manager_Num_Products','Manager_Business2', 'Manager_Num_Products2'])

#__Test Dataset__

UNI_numerical(test, ['Manager_Num_Application','Manager_Num_Coded', 'Manager_Business'])

UNI_numerical(test, ['Manager_Num_Products','Manager_Business2', 'Manager_Num_Products2'])

#1.3.2. Outliers

def outliers_detection(df, num_feats):
    
    plt.figure(figsize=(7*len(num_feats), 4))
    
    for index, feat in enumerate(num_feats) :
        plt.subplot(1, 3, index+1)
        sns.boxplot(train[feat])
        plt.xlabel('{}'.format(feat), fontsize = 20)
    
    return

#__Train Dataset__

outliers_detection(train, ['Manager_Num_Application','Manager_Num_Coded', 'Manager_Business'])

outliers_detection(train, ['Manager_Num_Products','Manager_Business2', 'Manager_Num_Products2'])

#__Test Dataset__

outliers_detection(train, ['Manager_Num_Application','Manager_Num_Coded', 'Manager_Business'])

outliers_detection(train, ['Manager_Num_Products','Manager_Business2', 'Manager_Num_Products2'])

#1.4. Correlations

numerical_features = ["Manager_Num_Application", "Manager_Num_Coded", "Manager_Business", "Manager_Num_Products",
"Manager_Business2", "Manager_Num_Products2", "Business_Sourced"]

correlation_train = train[numerical_features].corr(method = 'pearson')
plt.title("Train Dataset Correlations", fontsize=20)
sns.heatmap(correlation_train, linewidth = 4, annot=True, cmap='plasma')

#The features **Manager_Business** and **Manager_Business2** are highly coorelated. Similarly a high correlation is observed between **Manager_Num_Products** and **Manager_Num_Products2**. <br>
#In order to remove multi-colinearity the columns **Manager_Business2** and **Manager_Num_Products2** will be dropped.


#As expected there will be a **strong correlation** between **Manager_Num_Products** and **Manager_Business**. 


# **As the number of products sold increases the amount of business sourced will also increase.**

plt.figure(figsize=(8,6))
sns.scatterplot(x='Manager_Business', y='Manager_Num_Products', data=train, hue='Business_Sourced')

#1.5. Categorical Distribution

categorical_features = ["Applicant_Gender", "Applicant_Marital_Status","Applicant_Occupation", 
                    "Applicant_Qualification", "Manager_Joining_Designation","Manager_Current_Designation"
                    ,"Manager_Grade","Manager_Status","Manager_Gender"]

fig, axs = plt.subplots(ncols=3, nrows=3, figsize=(20,20))
plt.subplots_adjust(right=1.5, wspace=0.4, hspace=0.4)

for i , col in enumerate(categorical_features) :
    plt.subplot(3, 3, i+1)
    # Distribution of Is Lead in feature
    sns.countplot(x= col, data = train, hue='Business_Sourced')
    plt.xlabel('')
    plt.ylabel('')
    plt.tick_params(axis='x', labelsize=20)
    plt.tick_params(axis='y', labelsize=20)
    plt.legend(loc='upper right', prop={'size': 20})
    plt.title('Count of Business_Sourced in {}'.format(col), size=20, y=1.05)
        
plt.show()

plt.figure(figsize=(20,10))
sns.countplot(x='Applicant_Qualification', data=train, hue='Business_Sourced')
plt.tick_params(axis='x', labelrotation=45)
plt.title("Count of Business Sourced per Applicant Qualification Group", fontsize=25)
plt.legend(loc='upper right')
plt.xlabel("Applicant's Qualification")

#1.6. DataType of features

#The data type of the date columns **'Application_Receipt_Date', 'Applicant_BirthDate', 'Manager_DoB', 'Manager_DOJ'** are not in datetime. 
#The datatype conversion of the columns needs to be done in the data cleaning step.

train.info()

train['Application_Receipt_Date'] = pd.to_datetime(train['Application_Receipt_Date'])
test['Application_Receipt_Date'] = pd.to_datetime(test['Application_Receipt_Date'])

train.info()

test.info()

# 1.7. Insights around the features

# Dictionary containing the month names

Calender_Month_Name = {
    1: 'Jan',
    2: 'Feb',
    3: 'Mar',
    4: 'Apr',
    5: 'May',
    6: 'Jun',
    7: 'Jul',
    8: 'Aug',
    9: 'Sep',
    10: 'Oct',
    11: 'Nov',
    12: 'Dec'
}

#1.7.1. Number of applications received per Application_Receipt_Date

# **CONCLUSION**
#The peak number of applications were received in the month of **May, 2007**. In initial months the number of applicatins received was low. However the number increased in the subsequent months.**The a huge bulk of applications are received in the months starting from July till December in both the years of 2007 and 2008**

# Number of applications received per day. (More Granular data at day level)

num_applications_per_day = train.groupby(['Application_Receipt_Date'])['ID'].count().reset_index()
num_applications_per_day['App_Year'] = num_applications_per_day['Application_Receipt_Date'].dt.year
num_applications_per_day['App_Month'] = num_applications_per_day['Application_Receipt_Date'].dt.month
num_applications_per_day['App_Day'] = num_applications_per_day['Application_Receipt_Date'].dt.day
num_applications_per_day['App_Date_Num'] = num_applications_per_day['App_Year']*10000+num_applications_per_day['App_Month']*100+num_applications_per_day['App_Day']

num_applications_per_day.head()
num_applications_per_day.tail()

# Number of applications received per month-year. (More Granular data at day level)

num_app_month_year = num_applications_per_day.groupby(['App_Year', 'App_Month'])['ID'].sum().reset_index()
num_app_month_year['Month_Year'] = num_app_month_year['App_Year']*100+num_app_month_year['App_Month']
num_app_month_year

plt.figure(figsize=(12,8))
sns.barplot(x="Month_Year", y="ID", data=num_app_month_year)
plt.title("Number of Applications received per Month", fontsize=15)
plt.xlabel("Month - Year")
plt.xticks(rotation=45)
plt.show()

# Lookup table for plotting each year, month then day level data

app_year_month = num_applications_per_day[['App_Year', 'App_Month']].drop_duplicates()
app_year_month.shape

# Position of the plots for 16 months data covering both 2007 and 2008...

plot_locs = [[0,0], [0,1], [0,2], [0,3],
             [1,0], [1,1], [1,2], [1,3],
             [2,0], [2,1], [2,2], [2,3],
             [3,0], [3,1], [3,2], [3,3]]

fig, axs = plt.subplots(ncols=4, nrows=4, figsize=(30,30))
fig.subplots_adjust(wspace=0.2,  hspace=0.2)

print("Number of applications received at a day level")
print()

for i , row in enumerate(app_year_month.iterrows()):
    
    year = row[1][0]
    month = row[1][1]
    
    num_app_at_year_month = num_applications_per_day[
        (num_applications_per_day['App_Year']==year) & (num_applications_per_day['App_Month']==month)]
    
    plot_x = plot_locs[i][0]
    plot_y = plot_locs[i][1]
    
    sns.barplot(x="App_Day", y="ID", data=num_app_at_year_month, ax=axs[plot_x][plot_y])
    axs[plot_x][plot_y].set_title("{}, {}".format(Calender_Month_Name.get(month),year), fontsize=20)
    axs[plot_x][plot_y].tick_params(axis='x', labelrotation=45, labelsize=12)
    axs[plot_x][plot_y].set_ylabel(" ")
    axs[plot_x][plot_y].set_xlabel("Application Receipt Day")

#We have an idea about the number of applications received per day and the trend of the volumne of applications received throughout the year.<br> 

#1.7.2. Number of Products sold by the Manager in last 3 months at the time of Application_Receipt_Date

# **AIM - To investigate the trend of the number of products sold by the Manager in the last 3 months at the time of the application receipt date, to get an idea of the sucessfull and relatively failure months of business.** 

Sales = train.groupby(['Application_Receipt_Date', 'Business_Sourced'])['Manager_Num_Products'].sum().reset_index()

# Year, Month and Day extraction from Application_Receipt_Date.
Sales['App_Year'] = Sales['Application_Receipt_Date'].dt.year
Sales['App_Month'] = Sales['Application_Receipt_Date'].dt.month
Sales['App_Day'] = Sales['Application_Receipt_Date'].dt.day
Sales['App_Date_Num'] = Sales['App_Year']*10000+Sales['App_Month']*100+Sales['App_Day']
Sales['App_Day'] = Sales['App_Day'].astype('object')
Sales.head()

# Position of the plots for 16 months data covering both 2007 and 2008...

plot_locs = [[0,0], [0,1], [0,2], [0,3],
             [1,0], [1,1], [1,2], [1,3],
             [2,0], [2,1], [2,2], [2,3],
             [3,0], [3,1], [3,2], [3,3]]

fig, axs = plt.subplots(ncols=4, nrows=4, figsize=(30,30))
fig.subplots_adjust(wspace=0.2,  hspace=0.2) 


for i , row in enumerate(app_year_month.iterrows()):
    
    year = row[1][0]
    month = row[1][1]
    
    sales_at_year_month = Sales[(Sales['App_Year']==year) & (Sales['App_Month']==month)]
    
    plot_x = plot_locs[i][0]
    plot_y = plot_locs[i][1]
    
    sns.lineplot(x="App_Day", y="Manager_Num_Products", data=sales_at_year_month, hue="Business_Sourced", ax=axs[plot_x][plot_y])
    axs[plot_x][plot_y].set_title("{}, {}".format(Calender_Month_Name.get(month),year), fontsize=20)
    axs[plot_x][plot_y].tick_params(axis='x', labelsize=10)
    axs[plot_x][plot_y].set_xlabel("Application Receipt Day")
    axs[plot_x][plot_y].set_ylabel(" ")

# **CONCLUSION**
#Initially in the period of Apr - Aug 2007, the number of products sold where business was sourced is very less than the times when the business was not sourced. The number of products sold where business was sourced started to increase in September, 2007.<br>

#The difference between the number of products sold between busniess sourced and non-soucred gradually decreased and this trend continued till March, 2008. There were instances where Number of products sold when business was sourced is more than that when not sourced.<br>

#1.7.3. Status of applications per Application_Receipt_Date

#**AIM - To investigate each application received throughout the time period, whether the agent was able to source business within 3 months after his / her training.** 

# Business_Sourced = 1 for Business Sourced and 0 for not sourced

Application_0_1 = train[['ID', 'Application_Receipt_Date', 'Business_Sourced']]

# Calculating the sequence / record row number in which the application was received for each Application Receipt Date.

Application_0_1 = pd.concat([Application_0_1, Application_0_1.groupby(['Application_Receipt_Date']).cumcount()+1], 
                            axis=1)
Application_0_1.rename(columns={0: "App_Order"}, inplace=True)

# Year, Month and Day extraction from Application_Receipt_Date.

Application_0_1['App_Year'] = Application_0_1['Application_Receipt_Date'].dt.year
Application_0_1['App_Month'] = Application_0_1['Application_Receipt_Date'].dt.month
Application_0_1['App_Day'] = Application_0_1['Application_Receipt_Date'].dt.day
Application_0_1['App_Date_Num'] = Application_0_1['App_Year']*10000+Application_0_1['App_Month']*100+Application_0_1['App_Day']

Application_0_1.head()

# Position of the plots 16 months data covering both 2007 and 2008...

plot_locs = [[0,0], [0,1], [0,2], [0,3],
             [1,0], [1,1], [1,2], [1,3],
             [2,0], [2,1], [2,2], [2,3],
             [3,0], [3,1], [3,2], [3,3]]

fig, axs = plt.subplots(ncols=4, nrows=4, figsize=(30,30))
fig.subplots_adjust(wspace=0.2,  hspace=0.2) 


for i , row in enumerate(app_year_month.iterrows()):
    
    year = row[1][0]
    month = row[1][1]
    
    app_at_year_month = Application_0_1[(Application_0_1['App_Year']==year) & (Application_0_1['App_Month']==month)]
    
    plot_x = plot_locs[i][0]
    plot_y = plot_locs[i][1]
    
    sns.scatterplot(x="App_Order", y="App_Day", data=app_at_year_month, hue="Business_Sourced", ax=axs[plot_x][plot_y])
    axs[plot_x][plot_y].set_title("{}, {}".format(Calender_Month_Name.get(month),year), fontsize=20)
    axs[plot_x][plot_y].tick_params(axis='x', labelsize=20)
    axs[plot_x][plot_y].set_xlabel(" ")
    axs[plot_x][plot_y].set_ylabel("Application Receipt Day")

# **CONCLUSION** A trend is captured. For a particular day the agent's application which was received first or relatively at the beginning of the day was able to source business within 3 months post 7 day training. This pattern is observed across all the 16 months of the train dataset. 


#1.7.4. Business from Category A advisor

#The business from **`Category A advisor`** will be the difference between the **`Manager_Business`** and **`Manager_Business2`** since,
#- **`Manager_Business`** is the amount of business sourced by the manager in last 3 months, 
#- **`Manager_Business2`** is the amount of business sourced by the manager in last 3 months excluding business from their Category A advisor

#The number of products sold by **`Category A advisor`** in the last 3 months will be the difference between the **`Manager_Num_Products`** and **`Manager_Num_Products2`** since,
#- **`Manager_Num_Products`** is Number of products sold by the manager in last 3 months
#- **`Manager_Num_Products2`** is the number of products sold by the manager in last 3 months excluding business from their Category A advisor

Category_A_Business = train[['Manager_Business', 'Manager_Business2', 'Manager_Num_Products', 'Manager_Num_Products2']]

Category_A_Business['Num_Products_Category_A'] = Category_A_Business['Manager_Num_Products'] - Category_A_Business['Manager_Num_Products2']
Category_A_Business['Business_Sourced_Category_A'] = Category_A_Business['Manager_Business'] - Category_A_Business['Manager_Business2']

Category_A_Business.head()

Category_A_Business[['Business_Sourced_Category_A', 'Num_Products_Category_A']].describe()

Category_A_Business[(Category_A_Business['Manager_Business'] < Category_A_Business['Manager_Business2'])]

#**CONCLUSION-** The features **Manager_Business** and **Manager_Business2** are highly coorelated. Similarly a high correlation is observed between **Manager_Num_Products** and **Manager_Num_Products2**. <br>

#In order to remove multi-colinearity the columns **Manager_Business2** and **Manager_Num_Products2** will be dropped.

#2. DATA PREPROCESSING 

# 19 features out of 23 had missing values.


# The Arbitray Value imputation is done for handling missing values in the numerical, categorical and date columns / features.


# Datatype conversion - The date columns were converted to proper datetime data type.


# Irrelevent features were dropped from train and test datasets.

train.isnull().sum()

#2.1. Missing Value Imputation

# Filling missing values of numerical features with an arbitrary value.

numerical_cols = ['Manager_Num_Application', 'Manager_Num_Coded', 'Manager_Business' , 'Manager_Num_Products', 
            'Manager_Business2', 'Manager_Num_Products2', 'Applicant_City_PIN']

for col in numerical_cols :
    train[col] = train[col].fillna(-999)
    test[col] = test[col].fillna(-999)

# Filling missing values of date features with an arbitrary value.

date_cols = ['Application_Receipt_Date', 'Applicant_BirthDate', 'Manager_DoB', 'Manager_DOJ']

for date_col in date_cols :
    train[date_col] = train[date_col].fillna("1/1/1900")
    test[date_col] = test[date_col].fillna("1/1/1900")

# Filling missing values of categorical features with an arbitrary value.

agent_manager_details = ['Applicant_City_PIN', 'Applicant_Marital_Status', 'Applicant_Occupation',
                         'Applicant_Qualification', 'Manager_Joining_Designation', 'Manager_Current_Designation',
                         'Manager_Grade', 'Manager_Status', 'Manager_Gender']

for feat in agent_manager_details :
    train[feat] = train[feat].fillna("Missing")
    test[feat] = test[feat].fillna("Missing")

train['Applicant_Gender'] = train['Applicant_Gender'].fillna('Gender_NA')
test['Applicant_Gender'] = test['Applicant_Gender'].fillna('Gender_NA')

#2.2. Data Type Conversion

# Converting date features from object to datetime to create new features from them in the feature engineering step.

date_cols = ['Applicant_BirthDate', 'Manager_DoB', 'Manager_DOJ']

for date_col in date_cols :
    train[date_col] = pd.to_datetime(train[date_col])
    test[date_col] = pd.to_datetime(test[date_col])

# Checking for any remaining features with missing values.

train.isnull().sum()

train.info()

#3. Feature Engineering

# 4 extra numerical features were created :
#1. Agent_Age : The age of the Applicant / agent as on Application Receipt Date.
#2. Manager_Age : The age of the Manager as on Application Receipt Date.
#3. Manager_Exp : The work experience of Manager in the company.
#4. App_Order_Percent : Percentile of the position of the Application Received calculated at a daily level.

#- The categorical features (Applicant_Gender, Applicant_Occupation) were One Hot Encoded.

#- (Manager_Joining_Designation, Manager_Current_Designation) were Label Encoded.

#Creating a function in order to build distribution plots in both train and test dataset for 
#3.1. Agent Age
#3.2. Manager Age
#3.3. Manager's Experience

def distribution_plot(feat):
    
    fig, axs = plt.subplots(nrows=1, ncols=2, figsize=(8,5))
    fig.subplots_adjust(right=1.5)

    sns.distplot(x=train[feat], ax=axs[0])
    axs[0].set_title("{} Distribution in train dataset".format(feat), fontsize=15)

    sns.distplot(x=test[feat], ax=axs[1])
    axs[1].set_title("{} Distribution in test dataset".format(feat), fontsize=15)

    plt.show()
    
    return

#3.1. Age of the Applicant / Agent

train['Agent_Age'] = ((train['Application_Receipt_Date'] - train['Applicant_BirthDate'])/np.timedelta64(1, 'D'))/365
test['Agent_Age'] = ((test['Application_Receipt_Date'] - test['Applicant_BirthDate'])/np.timedelta64(1, 'D'))/365

distribution_plot("Agent_Age")

#3.2. Age of the Manager

train['Manager_Age'] = ((train['Application_Receipt_Date'] - train['Manager_DoB'])/np.timedelta64(1, 'D'))/365
test['Manager_Age'] = ((test['Application_Receipt_Date'] - test['Manager_DoB'])/np.timedelta64(1, 'D'))/365

distribution_plot("Manager_Age")

#3.3. Manager's Experience in FinMan

train['Manager_Exp'] = ((train['Application_Receipt_Date'] - train['Manager_DOJ'])/np.timedelta64(1, 'D'))/365
test['Manager_Exp'] = ((test['Application_Receipt_Date'] - test['Manager_DOJ'])/np.timedelta64(1, 'D'))/365

distribution_plot("Manager_Exp")

#3.4. App_Order_percent

#Creating a function to check the relationship of the Application sequence w.r.t Application Receipt and sourced business
def App_Seq(df):
    
    cols= ['ID', 'Application_Receipt_Date']
    
    Applications = df[cols]
    Applications = pd.concat([Applications, Applications.groupby(['Application_Receipt_Date']).cumcount()+1], axis=1)
    Applications.rename(columns={0: "App_Order"}, inplace=True)
    max_app_seq = Applications.groupby(['Application_Receipt_Date'])['App_Order'].max().reset_index()
    min_app_seq = Applications.groupby(['Application_Receipt_Date'])['App_Order'].min().reset_index()

    max_min_seq = pd.merge(max_app_seq, min_app_seq , how='inner' , on='Application_Receipt_Date')
    max_min_seq.rename(columns={'App_Order_x' : 'Max_Seq', 'App_Order_y' : 'Min_Seq'}, inplace=True)

    max_min_seq['Range_Seq'] = max_min_seq['Max_Seq'] - max_min_seq['Min_Seq']
    Applications = pd.merge(Applications, max_min_seq, how='inner' , on='Application_Receipt_Date')
    
    Applications['App_Order_percent'] = (Applications['App_Order'] - Applications['Min_Seq'])/Applications['Range_Seq']
    
    
    df = pd.merge(df, Applications.drop(columns=['App_Order', 'Application_Receipt_Date', 'Max_Seq', 'Min_Seq',
                                                 'Range_Seq']) , how='inner', on='ID')
    
    return df, Applications

train, App_data_train = App_Seq(train)
test, App_data_test = App_Seq(test)

train.head()

#**CONCLUSION- The Applications placed at first in the sequence per Application Receipt Date are more likely to have sourced business**

print(train[train['Business_Sourced']==1]['App_Order_percent'].mean())

#__Handling Missing Values in App_Order_Percent Feature__

train[train['App_Order_percent'].isnull()]

test[test['App_Order_percent'].isnull()]

train['App_Order_percent'] = train['App_Order_percent'].fillna(0.00)
test['App_Order_percent'] = test['App_Order_percent'].fillna(0.00)

train.columns

train.info()

#3.5. Categorical Encodings

#3.5.1. Applicant Gender

train['Applicant_Gender'].unique()

train = pd.concat([train, pd.get_dummies(train['Applicant_Gender'], drop_first=True)], axis=1)
test = pd.concat([test, pd.get_dummies(test['Applicant_Gender'], drop_first=True)], axis=1)

train.info()

train.drop(columns=['Applicant_Gender'], inplace=True)
test.drop(columns=['Applicant_Gender'], inplace=True)

#3.5.2. Manager Joining Designation

# Label Encoding of Joining and Current Designation Levels

Designation_Level = {
    'Level 1' : 1, 
    'Level 2' : 2, 
    'Level 3' : 3,
    'Level 4' : 4,
    'Level 5' : 5,
    'Level 6' : 6,
    'Level 7' : 7,
    'Missing' : -1, 
    'Other' : -1
}

train['Manager_Joining_Designation'] = train['Manager_Joining_Designation'].map(Designation_Level)
test['Manager_Joining_Designation'] = test['Manager_Joining_Designation'].map(Designation_Level)

#3.5.3. Manager Current Designation

train['Manager_Current_Designation'] = train['Manager_Current_Designation'].map(Designation_Level)
test['Manager_Current_Designation'] = test['Manager_Current_Designation'].map(Designation_Level)

#3.5.4. Applicant Occupation

train['Applicant_Occupation'].unique()

train = pd.concat([train, pd.get_dummies(train['Applicant_Occupation'])] , axis=1)
train.head()

test = pd.concat([test, pd.get_dummies(test['Applicant_Occupation'])] , axis=1)

#4. Droping Irrelevent Columns

irrelevent_cols = ["ID", "Applicant_BirthDate", "Applicant_Marital_Status", "Applicant_Qualification", "Manager_DOJ",
"Manager_Grade", "Manager_Status", "Manager_Gender", "Manager_DoB", "Manager_Business2", "Manager_Num_Products2",
                  "Application_Receipt_Date", "Applicant_Occupation"]

train.drop(columns=irrelevent_cols, inplace=True)

df_test = test.copy()

test.drop(columns=irrelevent_cols, inplace=True)

train.info()

test.info()

plt.figure(figsize=(15,15))
correlation = train.corr(method="pearson")
sns.heatmap(correlation, annot=True, cmap="viridis")

#5. Modelling

# Spliting target variable from train dataset

X = train.drop(columns=['Business_Sourced'])
y = train[['Business_Sourced']]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

#5.1. XGBoost Model

xgb_model = XGBClassifier(objective="binary:logistic", n_estimators=200, learning_rate=0.05, max_depth=6, 
                          subsample=0.75, colsample_bytree=0.8, min_child_weight=1, n_jobs=-1, random_state=42,
                          eval_metric="auc") 

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

xgb_model.fit(X_train,y_train)
proba = xgb_model.predict_proba(X_train)[:,1]
train_score = roc_auc_score(y_train,proba)
cv_score = cross_val_score(xgb_model,X_train,y_train,scoring="roc_auc",verbose=2,cv =skf)

print('Train score : {}', train_score)
print('CV Scores : {}', cv_score)
print('Mean CV Scores : {}, Variance in CV Scores : {}'.format(cv_score.mean(), cv_score.std()))

xgb_model.fit(X_train,y_train)
proba = xgb_model.predict_proba(X_test)[:,1]
test_score = roc_auc_score(y_test,proba)
print('Test score : {}', test_score)

fig, ax = plt.subplots(figsize=(10,10))
plot_importance(xgb_model , ax=ax)

#In the XGBoost model, the top 5 features of importance are : 
#Agent_Age, App_Order_percent, Manager_Age, Applicant_City_PIN and Manager_Exp.

xgb_model.fit(X,y)

pred = xgb_model.predict_proba(test)[:,1]

submission = pd.DataFrame()
submission['ID'] = df_test['ID']
submission["Business_Sourced"] = pred

submission.to_csv('xgb_predictions_cv_Sami1.csv', index=False)

#5.2. Light Gradient Boosting Model

param_dist = {"n_estimators":sp_randint(40,80),
              "colsample_bytree":np.array([0.5,0.6,0.7,0.8,0.9,1]),
              "subsample":np.array([0.5,0.6,0.7,0.8,0.9,1]),
              "reg_lambda":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
              "reg_alpha":np.array([1e-5,1e-4,1e-3,1e-2,0.1,1,10,100]),
              "min_child_samples": sp_randint(25,65),
                "max_depth": sp_randint(1,20)}

clf_lgbm = LGBMClassifier(boosting_type = "gbdt",n_jobs =-1,random_state = 42)

# Randomized Search CV for finding the best parameters under roc_auc scoring

lgbm_cv = RandomizedSearchCV(clf_lgbm, param_distributions=param_dist,
                                   n_iter=50,cv=skf,scoring='roc_auc',random_state=42,verbose=1)

lgbm_cv.fit(X_train,y_train)

print('mean test scores',lgbm_cv.cv_results_['mean_test_score'])
print(lgbm_cv.cv_results_['mean_test_score'].mean(), lgbm_cv.cv_results_['mean_test_score'].std())

print('The best parameters {}'.format(lgbm_cv.best_params_))

lgb_clf_model = LGBMClassifier(colsample_bytree=0.4, max_depth=12, min_child_samples=49,
               n_estimators=59, random_state=42, reg_alpha=1.0,
               reg_lambda=0.001, subsample=1.0)

lgb_clf_model.fit(X_train,y_train)
proba = lgb_clf_model.predict_proba(X_train)[:,1]
train_score = roc_auc_score(y_train,proba)
cv_score = cross_val_score(lgb_clf_model,X_train,y_train,scoring="roc_auc",verbose=2,cv =skf)

print('Train score : {}', train_score)
print('CV Scores : {}', cv_score)
print('Mean CV Scores : {}, Variance in CV Scores : {}'.format(cv_score.mean(), cv_score.std()))

lgb_clf_model.fit(X_train,y_train)
proba = lgb_clf_model.predict_proba(X_test)[:,1]
test_score = roc_auc_score(y_test,proba)
print('Test score : {}', test_score)

lgb_clf_model.fit(X,y)

pred = lgb_clf_model.predict_proba(test)[:,1]
lgbm_submission = pd.DataFrame()
lgbm_submission['ID'] = df_test['ID']
lgbm_submission["Business_Sourced"] = pred

lgbm_submission.to_csv('lgbm_pred_cv_Sami3.csv', index=False)

feature_imp = pd.DataFrame(sorted(zip(lgb_clf_model.feature_importances_,X.columns)), columns=['Value','Feature'])

plt.figure(figsize=(10, 10))
sns.barplot(x="Value", y="Feature", data=feature_imp.sort_values(by="Value", ascending=False))
plt.title('LightGBM Features (avg over folds)')
plt.tight_layout()
plt.show()

#In the LightGBM model, the top 5 features of importance are : 
#App_Order_percent, Manager_Exp, Applicant_City_PIN, Office_PIN, Manager_Age and Agent_Age.

#5.3. Ada Boost Model

param_dist = {"n_estimators":sp_randint(40,100),
              "learning_rate":np.array([0.5,0.6,0.7,0.8,0.9,1])}

ada_boost_mdl = AdaBoostClassifier(random_state=42)

ada_boost_cv = RandomizedSearchCV(ada_boost_mdl, param_distributions=param_dist,
                                   n_iter=20,cv=8,scoring='roc_auc',random_state=42,verbose=1)

ada_boost_cv.fit(X_train,y_train)

print('mean test scores',ada_boost_cv.cv_results_['mean_test_score'])
print(ada_boost_cv.cv_results_['mean_test_score'].mean(), ada_boost_cv.cv_results_['mean_test_score'].std())

print('The best parameters {}'.format(ada_boost_cv.best_params_))

ada_boost = AdaBoostClassifier(learning_rate=0.9, n_estimators=60, random_state=42)

ada_boost.fit(X_train,y_train)

proba = ada_boost.predict_proba(X_test)[:,1]
test_score = roc_auc_score(y_test,proba)

print('Train Score : ',test_score)

ada_boost.fit(X,y)

pred = ada_boost.predict_proba(test)[:,1]
ada_boost_submission = pd.DataFrame()
ada_boost_submission['ID'] = df_test['ID']
ada_boost_submission["Business_Sourced"] = pred

ada_boost_submission.to_csv('ada_boost_predictions_Sami1.csv', index=False)

# ACCEPTED MODEL -  Light Gradient Boosting (Least Variance in CV scores)